In [5]:
library('clusterProfiler')
library(org.Mm.eg.db)
library(ggplot2)
library(gprofiler2)

In [29]:
indir <- '/data2st1/junyi/output/atac0627/snregulation'
outdir <- '/data2st1/junyi/output/atac0627/snregulation/gost/'
df_important_TF <- read.csv(paste0(indir, '/TFtarget_analysis_important_TF.csv'))
df_grns <- list.files('/data1st1/yejun/pyscenic/grn/', pattern = 'adj_.*\\.tsv', full.names = TRUE)
overwrite <- FALSE

In [3]:
df_important_TF.unique <- unique(df_important_TF$TF)

In [35]:
df_important_TF.unique

[1] "Npdc1"   "Jund"    "Cux1"    "Bclaf1"  "Bmyc"    "Foxp1"   "Zeb1"   
 [8] "Nfe2l1"  "Ep300"   "Srebf2"  "Xbp1"    "Foxn3"   "Tef"     "Kdm5a"  
[15] "Smarca4" "Zmiz1"   "Bach2"   "Tcf4"    "Nr2c2"   "Taf1"    "Tcf12"  
[22] "Rfx3"    "Pou2f1"  "Atf2"    "Bcl11a"  "Nr3c1"   "Mef2a"   "Hmgn3"  
[29] "Supt20"  "Irf2"    "Junb"    "Pbx1"    "Creb3"   "Zmat4"   "Nr2f2"  
[36] "Esrra"   "Phf8"    "Klf13"   "Zfp148"  "Foxp2"   "Clock"   "Hdac2"  
[43] "Atf4"    "Esrrg"   "Cebpz"   "Arnt2"   "Arnt"    "Ctcf"    "Rora"   
[50] "Tcf7l2"  "Thra"    "Rfx7"    "Pbx3"    "Pknox2"  "Meis2"   "Kdm5b"  
[57] "Meis1"   "Sox5"    "Prdx5"   "Zfp846"  "Setbp1"  "Foxj3"   "Thrb"   
[64] "Dbp"     "Rcor1"   "Setdb1"  "Hlf"     "Mef2c"   "Sox11"   "Nf1"    
[71] "Egr1"    "Egr4"    "Fos"     "Runx2"   "Lef1"    "Zbtb16"  "Foxo1"

In [ ]:
# for file in df_grns[:5]:
#     region = file.split("adj_")[1].split("_")[0]
#     gender = file.split(".")[0].split("_")[-1]
#     celltype = file.split("adj_")[1].replace(f'_{gender}.tsv','')

#     # region = 'TH'
#     # gender = 'M'
#     # celltype = 'Tnc_Adgrf5_Glut'
#     df_grn = pd.read_csv(file, sep='\t')
# for loop to read GRN files
# subset 5 files for testing
df_grns <- df_grns[1:3]  # Adjust this line to read all files if needed
for (file in df_grns) {
  celltype <- gsub("^.*adj_([A-Za-z0-9_]+)_.*$", "\\1", file) 
  gender <-gsub(".*_([A-Za-z]+)\\.tsv", "\\1", file)
  region <- gsub("^.*adj_([A-Za-z0-9]+)_([A-Za-z0-9_]+).*$", "\\1", file) 

  # if file already exists and overwrite is FALSE, skip
  df_grn <- read.table(file, header = TRUE, sep = "\t")

  # filter the GRN dataframe to only include important TFs
  df_grn <- df_grn[df_grn$TF %in% df_important_TF.unique, ]

  for (tf in unique(df_grn$TF)) {

      if (file.exists(paste(outdir, "GO_enrichment",celltype,gender,tf,".csv",sep = "_")) && !overwrite) {
        next
      }
        tryCatch(
        {
          subset_df <- df_grn[df_grn$TF == tf, ]
          gene_type_label = "gene"
          gene_list <- unique(subset_df$target)
          if(length(gene_list) > 0) {
      
            gostres <- gost(query = gene_list,
                            organism = "mmusculus",
                            sources = c("GO:MF", "GO:CC", "GO:BP"),
                            ordered_query = TRUE,
                            significant = TRUE,
                            user_threshold = 0.05,
                            correction_method = "g_SCS",
                            evcodes = TRUE)
            
            if(!is.null(gostres$result) && nrow(gostres$result) > 0) {
              res <- gostres$result
              res$celltype <- celltype
              res$gene_type_group <- gene_type_label


              for (col_name in names(res)) {
                if (class(res[[col_name]]) == "list") {
                  # Convert list elements to a single string, e.g., comma-separated
                  res[[col_name]] <- sapply(res[[col_name]], function(x) paste(x, collapse = "/"))
              }}
              write.csv(res, file = paste(outdir, "GO_enrichment",celltype,gender,tf,".csv",sep = "_"), row.names = FALSE)

              #all_results[[celltype]] <- res
            } else {
              cat("No enrichment results for gene_type =", gene_type_label, "celltype =", celltype, "\n")
            }
          } else {
            cat("No genes for gene_type =", gene_type_label, "celltype =", celltype, "\n")
          }
        }, 
        error = function(e) {
          print(paste("Error in subsetting data for", celltype, gender))
      })
  }
  
}

In [ ]:
gene_type_label = "gene"
all_results <- list()

gene_list <- unique(subset_df$target)
    
    if(length(gene_list) > 0) {
      
      gostres <- gost(query = gene_list,
                      organism = "mmusculus",
                      sources = c("GO:MF", "GO:CC", "GO:BP"),
                      ordered_query = TRUE,
                      significant = TRUE,
                      user_threshold = 0.05,
                      correction_method = "g_SCS",
                      evcodes = TRUE)
      
      if(!is.null(gostres$result) && nrow(gostres$result) > 0) {
        res <- gostres$result
        res$celltype <- celltype
        res$gene_type_group <- gene_type_label

        #all_results[[celltype]] <- res
      } else {
        cat("No enrichment results for gene_type =", gene_type_label, "celltype =", celltype, "\n")
      }
    } else {
      cat("No genes for gene_type =", gene_type_label, "celltype =", celltype, "\n")
    }
  
  
#   # 合并有效结果
#   valid_results <- all_results[sapply(all_results, function(x) !is.null(x) && nrow(x) > 0)]
#   if(length(valid_results) > 0) {
#     do.call(rbind, valid_results)
#   } else {
#     cat("No enrichment results generated overall for gene_type =", gene_type_label, "\n")
#     NULL
#   }

# valid_results$parents_str <- sapply(valid_results$parents, function(x) {
#   if(length(x) == 0) return(NA)  # 如果为空，写NA
#   paste(x, collapse = ";")
# })

# valid_results$parents <- valid_results$parents_str
# valid_results$parents_str <- NULL     

In [ ]:
#读取 DEG 数据 
data = read.csv("all.markers_anno.csv")

data <- data[order(-abs(data$avg_log2FC)), ]

data <- subset(data, 
              !grepl("^mt-", gene, ignore.case = TRUE) &
                !grepl("^Rp", gene))


data$celltype = data$cluster


###############################################################################
## 复用富集函数
perform_enrichment <- function(sub_data, gene_type_label) {
  all_results <- list()
  
  samples <- unique(sub_data$celltype)  # 按celltype分组
  
  for(smp in samples) {
    cat("Processing gene_type =", gene_type_label, "celltype =", smp, "\n")
    
    subset_data <- subset(sub_data, celltype == smp & !is.na(gene))
    gene_list <- unique(subset_data$gene)
    
    if(length(gene_list) > 0) {
      
      gostres <- gost(query = gene_list,
                      organism = "mmusculus",
                      sources = c("GO:MF", "GO:CC", "GO:BP"),
                      ordered_query = TRUE,
                      significant = TRUE,
                      user_threshold = 0.05,
                      correction_method = "g_SCS",
                      evcodes = TRUE)
      
      if(!is.null(gostres$result) && nrow(gostres$result) > 0) {
        res <- gostres$result
        res$celltype <- smp
        res$gene_type_group <- gene_type_label
        
        all_results[[smp]] <- res
      } else {
        cat("No enrichment results for gene_type =", gene_type_label, "celltype =", smp, "\n")
      }
    } else {
      cat("No genes for gene_type =", gene_type_label, "celltype =", smp, "\n")
    }
  }
  
  # 合并有效结果
  valid_results <- all_results[sapply(all_results, function(x) !is.null(x) && nrow(x) > 0)]
  if(length(valid_results) > 0) {
    do.call(rbind, valid_results)
  } else {
    cat("No enrichment results generated overall for gene_type =", gene_type_label, "\n")
    NULL
  }
}

go_results <- perform_enrichment(data, "gene")



go_results$parents <- go_results$parents_str
go_results$parents_str <- NULL

In [ ]:
# for file in df_grns[:5]:
#     region = file.split("adj_")[1].split("_")[0]
#     gender = file.split(".")[0].split("_")[-1]
#     celltype = file.split("adj_")[1].replace(f'_{gender}.tsv','')

#     # region = 'TH'
#     # gender = 'M'
#     # celltype = 'Tnc_Adgrf5_Glut'
#     df_grn = pd.read_csv(file, sep='\t')
# for loop to read GRN files
# subset 5 files for testing
for (file in df_grns) {
  celltype <- gsub("^.*adj_([A-Za-z0-9_]+)_.*$", "\\1", file) 
  gender <-gsub(".*_([A-Za-z]+)\\.tsv", "\\1", file)
  region <- gsub("^.*adj_([A-Za-z0-9]+)_([A-Za-z0-9_]+).*$", "\\1", file) 

  # if file already exists and overwrite is FALSE, skip
  df_grn <- read.table(file, header = TRUE, sep = "\t")

  # filter the GRN dataframe to only include important TFs
  df_grn <- df_grn[df_grn$TF %in% df_important_TF.unique, ]

  # for each tf in df_grn.TF.unique:
  for (tf in unique(df_grn$TF)) {

      if (file.exists(paste(outdir, "GO_enrichment_memento",celltype,gender,tf,".csv",sep = "_")) && !overwrite) {
        next
      }


        tryCatch(
        {
          print(paste("Processing:", celltype, gender))
          subset_df <- df_grn[df_grn$TF == tf, ]

          print(paste("Number of genes:", nrow(subset_df)))

          gene_entrez <- bitr(unique(subset_df$target), 
                              fromType = "SYMBOL", 
                              toType = "ENTREZID", 
                              OrgDb = "org.Mm.eg.db")
                              
          entrez_ids <- gene_entrez$ENTREZID
          ego <- enrichGO(gene = entrez_ids,
                          OrgDb = org.Mm.eg.db,
                          keyType = "ENTREZID",
                          ont = "ALL",  # "BP", "MF", "CC" 或 "ALL"
                          pvalueCutoff = 0.05,
                          qvalueCutoff = 0.05,
                          readable = TRUE)  # 结果中显示基因符号

          # 查看结果
          write.csv(ego, file = paste(outdir, "GO_enrichment_memento",celltype,gender,tf,".csv",sep = "_"), row.names = FALSE)
          p <- dotplot(ego, showCategory = 30)
          ggsave(paste(outdir, "GO_enrichment_memento_dot",celltype,gender,tf,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

          p<-barplot(ego, showCategory = 30)  # 条形图
          ggsave(paste(outdir, "GO_enrichment_memento_bar",celltype,gender,tf,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

          kk <- enrichKEGG(gene = entrez_ids,
                          organism = "mmu",  # 小鼠KEGG代码
                          pvalueCutoff = 0.05,
                          qvalueCutoff = 0.05)

          # 可视化
          p<-dotplot(kk, showCategory = 30)
          write.csv(kk, file = paste(outdir, "KEGG_enrichment_memento",celltype,gender,tf,".csv",sep = "_"), row.names = FALSE)
          ggsave(paste(outdir, "kegg_enrichment_memento_dot",celltype,gender,tf,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸
        }, 
        error = function(e) {
          print(paste("Error in subsetting data for", celltype, gender))
      })
  }
  
}


[1] "Processing: AMY_Ccdc3_Acvr1c_Glut F"
[1] "Number of genes: 5975"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$target), fromType = "SYMBOL", toType = "ENTREZID", :
“11.26% of input gene IDs are fail to map...”
Reading KEGG annotation online: "https://rest.kegg.jp/link/mmu/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/mmu"...



[1] "Processing: AMY_Ccdc3_Acvr1c_Glut F"
[1] "Number of genes: 2764"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$target), fromType = "SYMBOL", toType = "ENTREZID", :
“11.47% of input gene IDs are fail to map...”


[1] "Processing: AMY_Ccdc3_Acvr1c_Glut F"
[1] "Number of genes: 3915"


...

[1] "Processing: AMY_Ccdc3_Acvr1c_Glut M"
[1] "Number of genes: 2538"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$target), fromType = "SYMBOL", toType = "ENTREZID", :
“12.21% of input gene IDs are fail to map...”


[1] "Processing: AMY_Ccdc3_Acvr1c_Glut M"
[1] "Number of genes: 1896"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$target), fromType = "SYMBOL", toType = "ENTREZID", :
“13.34% of input gene IDs are fail to map...”
Warning message in rep(yes, length.out = len):
“'x' is NULL so the result will be NULL”


[1] "Error in subsetting data for AMY_Ccdc3_Acvr1c_Glut M"
